In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install -q peft bitsandbytes accelerate datasets trl

In [3]:
!pip install -U transformers

In [4]:
!gdown --id 1XRHSuhwv5v5CefCnM37Ia3o5nMu9P7jg

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1XRHSuhwv5v5CefCnM37Ia3o5nMu9P7jg
To: /kaggle/working/cleaned_no_duplicates.csv
100%|███████████████████████████████████████| 22.5M/22.5M [00:00<00:00, 131MB/s]


In [6]:
!gdown --id 1Z2KTMwrQXnZ_kZ2_Bp7r-o4DhUUlw7Zp

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1Z2KTMwrQXnZ_kZ2_Bp7r-o4DhUUlw7Zp
From (redirected): https://drive.google.com/uc?id=1Z2KTMwrQXnZ_kZ2_Bp7r-o4DhUUlw7Zp&confirm=t&uuid=a2bb4c4e-30d7-4860-bf9f-e2b19ba538d0
To: /kaggle/working/PhiUSIIL_TinyLlama_LoRA_Checkpoints.zip
100%|████████████████████████████████████████| 151M/151M [00:01<00:00, 80.3MB/s]


In [7]:
!unzip /kaggle/working/PhiUSIIL_TinyLlama_LoRA_Checkpoints.zip

Archive:  /kaggle/working/PhiUSIIL_TinyLlama_LoRA_Checkpoints.zip
replace PhiUSIIL_TinyLlama_LoRA_Checkpoints/checkpoint-6000/training_args.bin? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [ ]:
from huggingface_hub import login
login()

In [9]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import pandas as pd
from datasets import Dataset

2025-12-01 11:34:11.843053: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764588851.863588     126 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764588851.869772     126 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [10]:
CHECKPOINT_DIR = "./PhiUSIIL_TinyLlama_LoRA_Checkpoints"
RESUME_FROM_CHECKPOINT = True  # Đặt True để tiếp tục từ checkpoint

def find_latest_checkpoint(output_dir):
    """Tìm checkpoint mới nhất"""
    if not os.path.exists(output_dir):
        return None
    checkpoints = [d for d in os.listdir(output_dir) if d.startswith("checkpoint-")]
    if not checkpoints:
        return None
    latest = max(checkpoints, key=lambda x: int(x.split("-")[1]))
    return os.path.join(output_dir, latest)

In [11]:
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Quan trọng cho padding

# Load model với 8-bit quantization
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    device_map="auto",  # Tự động phân bổ lên 2 GPU
    torch_dtype=torch.float16
)

# Chuẩn bị model cho k-bit training
model = prepare_model_for_kbit_training(model)

# Cấu hình LoRA - Tối ưu hơn
lora_config = LoraConfig(
    r=16,                    # Tăng từ 8 -> 16 cho chất lượng tốt hơn
    lora_alpha=32,           # = 2*r
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],  # Thêm k_proj, o_proj
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

trainable params: 4,505,600 || all params: 1,104,553,984 || trainable%: 0.4079


In [24]:
#df = pd.read_csv("/kaggle/working/cleaned_no_duplicates.csv", sep=";",on_bad_lines="skip")

# Giảm dataset xuống 50% để chạy nhanh hơn (có thể điều chỉnh)
# Nếu muốn dùng full data, comment dòng dưới
#df = df.sample(frac=0.5, random_state=42).reset_index(drop=True)

df = df.sample(frac=1, random_state=42).reset_index(drop=True)
df["label_text"] = df["label"].map({0: "phishing", 1: "benign"})

dataset = Dataset.from_pandas(df)
train_test = dataset.train_test_split(test_size=0.1, seed=42)
train_data, val_data = train_test["train"], train_test["test"]

print(f"📊 Training samples: {len(train_data)}")
print(f"📊 Validation samples: {len(val_data)}")

📊 Training samples: 438339
📊 Validation samples: 48705


In [13]:
print("\n" + "="*50)
print("📊 PHÂN TÍCH CÂN BẰNG NHÃN")
print("="*50)

def check_label_balance(data, data_name):
    """Kiểm tra phân bố nhãn trong dataset"""
    labels = [item['label'] for item in data]
    
    total = len(labels)
    phishing_count = sum(1 for l in labels if l == 0)
    benign_count = sum(1 for l in labels if l == 1)
    
    phishing_pct = (phishing_count / total) * 100
    benign_pct = (benign_count / total) * 100
    
    print(f"\n🔍 {data_name}:")
    print(f"   Total: {total:,} samples")
    print(f"   ├─ Phishing: {phishing_count:,} ({phishing_pct:.2f}%)")
    print(f"   └─ Benign:   {benign_count:,} ({benign_pct:.2f}%)")
    
    # Tính tỷ lệ imbalance
    ratio = max(phishing_count, benign_count) / min(phishing_count, benign_count)
    print(f"   Imbalance Ratio: 1:{ratio:.2f}")
    
    if ratio > 1.5:
        print(f"   ⚠️  CẢNH BÁO: Dataset không cân bằng!")
    else:
        print(f"   ✅ Dataset khá cân bằng")
    
    return phishing_count, benign_count


📊 PHÂN TÍCH CÂN BẰNG NHÃN


In [ ]:
print("\n📁 TOÀN BỘ DATASET:")
total_phishing = sum(1 for l in df['label'] if l == 0)
total_benign = sum(1 for l in df['label'] if l == 1)
print(f"   Total: {len(df):,} samples")
print(f"   ├─ Phishing: {total_phishing:,} ({(total_phishing/len(df))*100:.2f}%)")
print(f"   └─ Benign:   {total_benign:,} ({(total_benign/len(df))*100:.2f}%)")

# Kiểm tra train/val
train_phish, train_benign = check_label_balance(train_data, "TRAIN SET")
val_phish, val_benign = check_label_balance(val_data, "VALIDATION SET")

print("\n" + "="*50)

In [25]:
df[df['label'].isna()]

,url,label,label_text


In [26]:
df['label'] = pd.to_numeric(df['label'], errors='coerce')
df = df.dropna(subset=['label'])
df['label'] = df['label'].astype(int)

In [28]:
Y1 = df['label']
X1 = df.drop(columns= ['url', 'label'])

In [17]:
X1.head(2)

,label_text
0,benign
1,phishing


In [18]:
Y1.head(300000)

0         1
1         0
2         1
3         1
4         0
         ..
300191    0
300192    1
300193    0
300194    0
300195    0
Name: label, Length: 300000, dtype: int64

In [19]:
X2 = df['url']
X2.head(2)

0                        https://zsassociates.com
1    http://itau-seguridad-validaciones.atsnx.com
Name: url, dtype: object

In [20]:
df.shape

(487044, 3)

In [21]:
# Sử dụng hàm describe() của pandas trên cột 'type' trong dataframe df.
# Hàm này sẽ trả về các thông tin thống kê mô tả của cột 'type',
# bao gồm: count (số lượng giá trị không null), unique (số lượng giá trị duy nhất),
# top (giá trị xuất hiện nhiều nhất), freq (tần suất xuất hiện của giá trị top)
# nếu cột 'type' là dạng object (categorical/text).
df["label"].describe()


count    487044.00000
mean          0.50781
std           0.49994
min           0.00000
25%           0.00000
50%           1.00000
75%           1.00000
max           1.00000
Name: label, dtype: float64

In [29]:
df["label"].value_counts()

label
1    247326
0    239718
Name: count, dtype: int64

In [30]:
def format_example(example):
    messages = [
        {"role": "system", "content": "You are a cybersecurity assistant that classifies URLs as phishing or benign."},
        {"role": "user", "content": f"Classify the following URL:\n{example['url']}"},
        {"role": "assistant", "content": example["label_text"]},
    ]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
    return {"text": text}

train_data = train_data.map(format_example, num_proc=4)  # Dùng 4 cores để xử lý song song
val_data = val_data.map(format_example, num_proc=4)

Map (num_proc=4):   0%|          | 0/438339 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/48705 [00:00<?, ? examples/s]

In [31]:
train_data[:2]

{'url': ['https://goblalture3onlperno3n.becglogalce2ma.repl.co/',
  'http://europa.eu/about-eu/agencies/regulatory_agencies_bodies/policy_agencies/berec/index_en.htm'],
 'label': [0, 1],
 'label_text': ['phishing', 'benign'],
 'text': ['<|system|>\nYou are a cybersecurity assistant that classifies URLs as phishing or benign.</s>\n<|user|>\nClassify the following URL:\nhttps://goblalture3onlperno3n.becglogalce2ma.repl.co/</s>\n<|assistant|>\nphishing</s>\n',
  '<|system|>\nYou are a cybersecurity assistant that classifies URLs as phishing or benign.</s>\n<|user|>\nClassify the following URL:\nhttp://europa.eu/about-eu/agencies/regulatory_agencies_bodies/policy_agencies/berec/index_en.htm</s>\n<|assistant|>\nbenign</s>\n']}

In [32]:
def tokenize(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        padding="max_length",
        max_length=256,
)

X_train = train_data.map(
    tokenize, 
    batched=True, 
    remove_columns=["text", "url", "label", "label_text"],
    num_proc=4
)

X_val = val_data.map(
    tokenize, 
    batched=True, 
    remove_columns=["text", "url", "label", "label_text"],
    num_proc=4
)

Map (num_proc=4):   0%|          | 0/438339 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/48705 [00:00<?, ? examples/s]

In [33]:
print(X_train[0])

{'input_ids': [1, 529, 29989, 5205, 29989, 29958, 13, 3492, 526, 263, 5094, 495, 8926, 20255, 393, 770, 11057, 24295, 408, 1374, 14424, 470, 3856, 647, 29889, 2, 29871, 13, 29966, 29989, 1792, 29989, 29958, 13, 2385, 1598, 278, 1494, 3988, 29901, 13, 991, 597, 29887, 711, 29880, 1997, 545, 29941, 265, 29880, 546, 1217, 29941, 29876, 29889, 19385, 29887, 1188, 284, 346, 29906, 655, 29889, 276, 572, 29889, 1111, 29914, 2, 29871, 13, 29966, 29989, 465, 22137, 29989, 29958, 13, 561, 14424, 2, 29871, 13, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 

In [34]:
X_train.save_to_disk("/kaggle/working/X_train8")
X_val.save_to_disk("/kaggle/working/X_test8")

Saving the dataset (0/2 shards):   0%|          | 0/438339 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/48705 [00:00<?, ? examples/s]

In [35]:
!zip -r /kaggle/working/X_test8.zip /kaggle/working/X_test8
!zip -r /kaggle/working/X_train8.zip /kaggle/working/X_train8

  adding: kaggle/working/X_test8/ (stored 0%)
  adding: kaggle/working/X_test8/state.json (deflated 39%)
  adding: kaggle/working/X_test8/dataset_info.json (deflated 61%)
  adding: kaggle/working/X_test8/data-00000-of-00001.arrow (deflated 97%)
  adding: kaggle/working/X_train8/ (stored 0%)
  adding: kaggle/working/X_train8/state.json (deflated 47%)
  adding: kaggle/working/X_train8/data-00001-of-00002.arrow (deflated 97%)
  adding: kaggle/working/X_train8/data-00000-of-00002.arrow (deflated 97%)
  adding: kaggle/working/X_train8/dataset_info.json (deflated 61%)


In [36]:
training_args = TrainingArguments(
    output_dir=CHECKPOINT_DIR,
    
    # Batch size & Gradient Accumulation
    per_device_train_batch_size=16,      # Tăng từ 8 -> 16
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=2,        # Giảm từ 4 -> 2
    # => Effective batch = 16 * 2 * 2 GPU = 64
    
    # Learning rate & Epochs
    learning_rate=3e-4,                   # Tăng LR để học nhanh hơn
    num_train_epochs=1,
    warmup_ratio=0.03,                    # Warmup 3% steps
    
    # Optimizer
    optim="adamw_torch_fused",            # Optimizer nhanh hơn
    
    # Precision
    fp16=True,
    bf16=False,
    
    # Logging & Checkpointing
    logging_steps=100,                    # Log thường xuyên hơn
    save_steps=500,                       # Lưu checkpoint mỗi 500 steps
    save_total_limit=3,                   # Chỉ giữ 3 checkpoint gần nhất
    
    # Evaluation
    eval_strategy="no",                   #Tắt eval
    load_best_model_at_end=False,         # Tắt để nhanh hơn
    
    # Performance
    dataloader_num_workers=4,             # Dùng 4 workers
    dataloader_pin_memory=True,
    gradient_checkpointing=True,          # Giảm memory, tăng tốc
    
    # Multi-GPU
    ddp_find_unused_parameters=False,
    
    # Misc
    report_to="none",
    remove_unused_columns=True,
    
    # Resume
    resume_from_checkpoint=find_latest_checkpoint(CHECKPOINT_DIR) if RESUME_FROM_CHECKPOINT else None,
)

In [37]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=X_train,
    eval_dataset=X_val,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

In [38]:
import numpy as np
import numpy.core.multiarray
import torch
import torch.serialization as ts

# 1) Allowlist numpy._reconstruct (cho đúng khuyến nghị của PyTorch)
ts.add_safe_globals([numpy.core.multiarray._reconstruct])

# 2) Patch torch.load để luôn dùng weights_only=False
_real_torch_load = torch.load

def torch_load_weights_only_false(*args, **kwargs):
    # Bất kể ai gọi torch.load, mình ép weights_only=False
    kwargs["weights_only"] = False
    return _real_torch_load(*args, **kwargs)

torch.load = torch_load_weights_only_false


In [39]:
print("\n" + "="*50)
print("🚀 BẮT ĐẦU TRAINING")
print("="*50)
print(f"✅ Số GPU được sử dụng: {torch.cuda.device_count()}")
print(f"✅ Trainable params: {model.print_trainable_parameters()}")

# Kiểm tra nếu có checkpoint
latest_checkpoint = find_latest_checkpoint(CHECKPOINT_DIR)
if RESUME_FROM_CHECKPOINT and latest_checkpoint:
    print(f"🔄 Tiếp tục từ checkpoint: {latest_checkpoint}")
    trainer.train(resume_from_checkpoint=latest_checkpoint)
else:
    print("🆕 Bắt đầu training mới")
    trainer.train()


🚀 BẮT ĐẦU TRAINING
✅ Số GPU được sử dụng: 2
trainable params: 4,505,600 || all params: 1,104,553,984 || trainable%: 0.4079
✅ Trainable params: None
🔄 Tiếp tục từ checkpoint: ./PhiUSIIL_TinyLlama_LoRA_Checkpoints/checkpoint-7000


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:181: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
7100,0.654400
7200,0.650900
7300,0.647100
7400,0.633500
7500,0.622900
7600,0.642100
7700,0.628800
7800,0.632000
7900,0.647100
8000,0.637600


/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:181: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:181: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:181: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:181: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during q

KeyboardInterrupt: 

In [40]:
!rm /kaggle/working/PhiUSIIL_TinyLlama_LoRA_Checkpoints.zip #Xóa file .zip cũ

In [42]:
import os
import shutil
from pathlib import Path

checkpoint_dir = "/kaggle/working/PhiUSIIL_TinyLlama_LoRA_Checkpoints"
output_dir = "/kaggle/working"

print("🗜️  Đang nén checkpoint...")

# Chuyển vào thư mục cha
os.chdir("/kaggle/working")

# Tạo file zip từ thư mục hiện tại
archive_name = "PhiUSIIL_TinyLlama_LoRA_Checkpoints"
shutil.make_archive(
    archive_name,           # Tên file (không cần .zip)
    'zip',                  # Format: 'zip', 'tar', 'gztar', 'bztar', 'xztar'
    root_dir='.',           # Thư mục gốc
    base_dir='PhiUSIIL_TinyLlama_LoRA_Checkpoints'  # Thư mục cần nén
)

print(f"✅ Đã nén xong: {archive_name}.zip")
print(f"📦 Kích thước: {os.path.getsize(f'{archive_name}.zip') / (1024**2):.2f} MB")

🗜️  Đang nén checkpoint...
✅ Đã nén xong: PhiUSIIL_TinyLlama_LoRA_Checkpoints.zip
📦 Kích thước: 144.41 MB


In [43]:
print("\n💾 Đang lưu model...")
trainer.save_model(f"{CHECKPOINT_DIR}/final_model")
tokenizer.save_pretrained(f"{CHECKPOINT_DIR}/final_model")

print("\n✅ HOÀN THÀNH!")
print(f"📁 Model đã được lưu tại: {CHECKPOINT_DIR}/final_model")


💾 Đang lưu model...

✅ HOÀN THÀNH!
📁 Model đã được lưu tại: ./PhiUSIIL_TinyLlama_LoRA_Checkpoints/final_model


In [44]:
print("\n" + "="*50)
print("🧪 KIỂM TRA MODEL")
print("="*50)

test_urls = [
    "https://www.google.com",
    "https://secure-login-verify-account-88392.web.app/"
]

for url in test_urls:
    messages = [
        {"role": "system", "content": "You are a cybersecurity assistant that classifies URLs as phishing or benign."},
        {"role": "user", "content": f"Classify the following URL:\n{url}"},
    ]
    
    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_dict=True,
        return_tensors="pt",
    ).to("cuda")
    
    outputs = model.generate(**inputs, max_new_tokens=10, temperature=0.1)
    result = tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True)
    
    print(f"URL: {url}")
    print(f"Prediction: {result}\n")

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Caching is incompatible with gradient checkpointing in LlamaDecoderLayer. Setting `past_key_values=None`.



🧪 KIỂM TRA MODEL


/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


URL: https://www.google.com
Prediction: ben < < < < < < < < <

URL: https://secure-login-verify-account-88392.web.app/
Prediction: ph < < < < < < < < <

